# Домашнее задание № 2. Мешок слов

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией, с токенизацией razdel.tokenize и с токенизацией+лемматизацией из mystem. Обучите классификатор (любой) с каждым из векторизаторов. Сравните метрики и выберите победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('labeled.csv')

Будем использовать обычный мешок слов. В токенизации с razdel и Mystem попробуем два подхода: с удалением стоп-слов и без него.

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import nltk
from nltk.corpus import stopwords
import razdel
from pymystem3 import Mystem

### 1. Готовим обучающую и тестовую выборки (с сохранением баланса классов)

In [5]:
print("Баланс классов:")
print(data['toxic'].value_counts(normalize=True))

Баланс классов:
toxic
0.0    0.66514
1.0    0.33486
Name: proportion, dtype: float64


In [6]:
X = data['comment']
y = data['toxic']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=y)

print(f"\nРазмер обучающей выборки: {len(X_train)}")
print(f"Размер тестовой выборки: {len(X_test)}")


Размер обучающей выборки: 11529
Размер тестовой выборки: 2883


### 2. Токенизаторы

Будем оставлять только алфавитные токены (слова). Встроенная регулярка в CountVectorizer (`token_pattern) оставляет ещё и цифры, поэтому у нас будет ещё и такое преимущество. Конечно, `token_pattern` можно и изменить, но зачем?

In [7]:
russian_stopwords = stopwords.words("russian")
mystem = Mystem()

In [8]:
# Токенизация с помощью razdel
def tokenize_razdel(text):
    return [token.text for token in razdel.tokenize(
        text) if token.text.isalpha()]

In [9]:
# Токенизация и лемматизация с Mystem
def tokenize_mystem(text):
    tokens = mystem.lemmatize(text)
    # Оставляем только слова
    tokens = [token for token in tokens if token.isalpha() and token.strip()]
    return tokens

### 3. Пайплайны: логистическая регрессия с по-разному настроенными векторизаторами

Тут удобнее всего использовать Sklearn pipeline, в который сразу передавать векторизатор и классификатор с нужными параметрами. По-хорошему надо бы задать нужные параметры через grid_search и выполнить всё одним скопом, но лень.

In [9]:
print("--- Метод 1: Default Tokenizer ---")

pipeline_default = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', LogisticRegression(random_state=42, max_iter=1000))
])

# Обучаем
pipeline_default.fit(X_train, y_train)

# Предсказываем и выводим метрики
preds_default = pipeline_default.predict(X_test)
print(classification_report(y_test, preds_default))

--- Метод 1: Default Tokenizer ---
              precision    recall  f1-score   support

         0.0       0.85      0.94      0.89      1918
         1.0       0.86      0.67      0.75       965

    accuracy                           0.85      2883
   macro avg       0.85      0.81      0.82      2883
weighted avg       0.85      0.85      0.85      2883



In [10]:
print("\n--- Метод 2: razdel.tokenize + Stopwords ---")

pipeline_razdel = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize_razdel, stop_words=russian_stopwords)),
    ('clf', LogisticRegression(random_state=42, max_iter=1000))
])

# Обучаем
pipeline_razdel.fit(X_train, y_train)

# Предсказываем и выводим метрики
preds_razdel = pipeline_razdel.predict(X_test)
print(classification_report(y_test, preds_razdel))


--- Метод 2: razdel.tokenize + Stopwords ---


/home/futyn/miniconda3/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


              precision    recall  f1-score   support

         0.0       0.83      0.94      0.88      1918
         1.0       0.85      0.61      0.71       965

    accuracy                           0.83      2883
   macro avg       0.84      0.78      0.79      2883
weighted avg       0.83      0.83      0.82      2883



In [11]:
print("\n--- Метод 3: Mystem lemmatize + Stopwords ---")

pipeline_mystem = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize_mystem, stop_words=russian_stopwords)),
    ('clf', LogisticRegression(random_state=42, max_iter=1000))
])

# Обучаем
pipeline_mystem.fit(X_train, y_train)

# Предсказываем и выводим метрики
preds_mystem = pipeline_mystem.predict(X_test)
print(classification_report(y_test, preds_mystem))


--- Метод 3: Mystem lemmatize + Stopwords ---


/home/futyn/miniconda3/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/futyn/miniconda3/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['весь', 'свой', 'это'] not in stop_words.
  warnings.warn(


              precision    recall  f1-score   support

         0.0       0.86      0.94      0.90      1918
         1.0       0.85      0.69      0.76       965

    accuracy                           0.86      2883
   macro avg       0.85      0.81      0.83      2883
weighted avg       0.85      0.86      0.85      2883



Хм, а если не удалять стоп-слова?

In [12]:
print("\n--- Метод 4: razdel.tokenize, сохраняем стопслова ---")

pipeline_razdel = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize_razdel)),
    ('clf', LogisticRegression(random_state=42, max_iter=1000))
])

# Обучаем
pipeline_razdel.fit(X_train, y_train)

# Предсказываем и выводим метрики
preds_razdel = pipeline_razdel.predict(X_test)
print(classification_report(y_test, preds_razdel))


--- Метод 4: razdel.tokenize, сохраняем стопслова ---


/home/futyn/miniconda3/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


              precision    recall  f1-score   support

         0.0       0.85      0.94      0.89      1918
         1.0       0.85      0.68      0.75       965

    accuracy                           0.85      2883
   macro avg       0.85      0.81      0.82      2883
weighted avg       0.85      0.85      0.85      2883



In [13]:
print("\n--- Метод 5: Mystem lemmatize, с сохранением стопслов ---")

pipeline_mystem = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize_mystem)),
    ('clf', LogisticRegression(random_state=42, max_iter=1000))
])

# Обучаем
pipeline_mystem.fit(X_train, y_train)

# Предсказываем и выводим метрики
preds_mystem = pipeline_mystem.predict(X_test)
print(classification_report(y_test, preds_mystem))


--- Метод 5: Mystem lemmatize, с сохранением стопслов ---


/home/futyn/miniconda3/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


              precision    recall  f1-score   support

         0.0       0.88      0.94      0.91      1918
         1.0       0.87      0.74      0.80       965

    accuracy                           0.88      2883
   macro avg       0.87      0.84      0.85      2883
weighted avg       0.88      0.88      0.87      2883



Что ж, однозначный победитель: токенизатор+лемматизатор Mystem с сохранением стоп-слов. Можно сделать два вывода:

1. Для русского языка лемматизация существенно повышает качество. Во-первых, потому что это флективный язык, и сведение множества форм слова к одной очень полезно. Во-вторых, потому что это уменьшает размер словаря и снижает размерность итоговых векторов, что упрощает задачу по обучению.
2. Для задачи определения сентимента стоп-слова важны, удалять их не надо. В принципе, логично, потому что там есть и отрицание, и всякие местоимения, которые часто используются в токсичных текстах.

А ещё дисбаланс классов всё-таки чувствуется, потому что самая низкая метрика во всех случаях — recall для положительного класса (токсичные тексты). Выходит, что модели всё же biased в сторону нетоксичных текстов и чаще предсказывают 0, потому что видели таких текстов больше.

## Задание 2 (3 балла)

Обучите 2 любых разных классификатора (используя алгоритмы из семинара). Используйте eng сабсет из `mteb/multi-hatecheck` в качестве датасета. 

Предскажите токсичность (`is_hateful`) для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов (по вероятности). Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к моделям:   
а) один классификатор должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров (можно ставить разные параметры tfidfvectorizer и countvectorizer)  
в) у классификатора должно быть задано вручную как минимум 2 параметра (по возможности)  
г)  f1 мера каждого из классификаторов должна быть минимум 0.60 

*random_seed, encoding, decode_error не считаются за параметры

In [15]:
import pandas as pd
import numpy as np
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

pd.set_option('display.max_colwidth', None)

### 1. Загружаем данные и смотрим на них, а также готовим

In [16]:
hatecheck = load_dataset('mteb/multi-hatecheck')
print(hatecheck)

DatasetDict({
    test: Dataset({
        features: ['functionality', 'text', 'is_hateful', 'lang'],
        num_rows: 40310
    })
})


У нас есть только тестовый датасет, так что придётся вручную бить его на сплиты и делать его часть тренировочным. В академических работах, кстати, это жёстко порицается.

In [17]:
# Фильтруем по английскому языку
eng_hatecheck = hatecheck.filter(lambda e: e["lang"] == 'eng')

# Конвертируем в pandas DataFrame для удобства
df = pd.DataFrame(eng_hatecheck['test'])

In [18]:
print(df.head())
print("\nИнформация о DataFrame:")
df.info()

print("\nБаланс классов:")
print(df['is_hateful'].value_counts(normalize=True))

       functionality                      text is_hateful lang
0  derog_neg_emote_h            I hate women.     hateful  eng
1  derog_neg_emote_h     I hate trans people.     hateful  eng
2  derog_neg_emote_h       I hate gay people.     hateful  eng
3  derog_neg_emote_h     I hate black people.     hateful  eng
4  derog_neg_emote_h  I hate disabled people.     hateful  eng

Информация о DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3728 entries, 0 to 3727
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   functionality  3728 non-null   object
 1   text           3728 non-null   object
 2   is_hateful     3728 non-null   object
 3   lang           3728 non-null   object
dtypes: object(4)
memory usage: 116.6+ KB

Баланс классов:
is_hateful
hateful        0.6875
non-hateful    0.3125
Name: proportion, dtype: float64


Класс, дисбаланс в другую сторону!

In [19]:
# Преобразуем текстовые метки в бинарные (1 для 'hateful', 0 для 'not_hateful')
df['label'] = df['is_hateful'].apply(lambda x: 1 if x == 'hateful' else 0)

### 2. Делаем тренировочную и тестовую выборки

In [20]:
X = df['text']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=y)

print(f"\nРазмер обучающей выборки: {len(X_train)}")
print(f"Размер тестовой выборки: {len(X_test)}")


Размер обучающей выборки: 2982
Размер тестовой выборки: 746


### 3. Модельки

Возьмём наивный Байесовский классификатор с мешком слов и мощный лес с TF-IDF. Будем использовать биграммы и триграммы.

In [21]:
pipeline_nb = Pipeline([
    ('vect', CountVectorizer(
        max_df=0.90,  # игнорируем слова, которые встречаются в > 90% документов
        min_df=3,  # игнорируем слова, которые встречаются < 3 раз
        ngram_range=(1, 2),  # включаем биграммы
        max_features=10000,  # ограничиваем словарь 10000 слов
        lowercase=True
    )),
    ('clf', MultinomialNB(
        alpha=0.1,
        fit_prior=False
    ))
])

In [22]:
print("--- Обучение: Модель 1 (Naive Bayes + CountVectorizer) ---")
pipeline_nb.fit(X_train, y_train)

preds_nb = pipeline_nb.predict(X_test)
print(classification_report(y_test, preds_nb))

--- Обучение: Модель 1 (Naive Bayes + CountVectorizer) ---
              precision    recall  f1-score   support

           0       0.90      0.82      0.86       233
           1       0.92      0.96      0.94       513

    accuracy                           0.92       746
   macro avg       0.91      0.89      0.90       746
weighted avg       0.91      0.92      0.91       746



In [23]:
pipeline_rf = Pipeline([
    ('vect', TfidfVectorizer(
        max_df=0.85,
        min_df=5,
        ngram_range=(1, 3),
        max_features=8000,
        sublinear_tf=True  # применяем 1 + log(tf) для сглаживания
    )),
    ('clf', RandomForestClassifier(
        n_estimators=200,
        max_depth=30,
        class_weight='balanced',  # пытаемся бороться с дисбалансом
        random_state=42,
        n_jobs=-1
    ))
])

In [24]:
print("--- Обучение: Модель 2 (Random Forest + TfidfVectorizer) ---")
pipeline_rf.fit(X_train, y_train)

preds_rf = pipeline_rf.predict(X_test)
print(classification_report(y_test, preds_rf))

--- Обучение: Модель 2 (Random Forest + TfidfVectorizer) ---
              precision    recall  f1-score   support

           0       0.97      0.81      0.88       233
           1       0.92      0.99      0.95       513

    accuracy                           0.93       746
   macro avg       0.94      0.90      0.92       746
weighted avg       0.93      0.93      0.93       746



### Анализируем топ-10

In [25]:
probs_nb = pipeline_nb.predict_proba(X_test)[:, 1]
probs_rf = pipeline_rf.predict_proba(X_test)[:, 1]

In [26]:
results_df = pd.DataFrame({
    'text': X_test,
    'true_label': y_test,
    'prob_nb': probs_nb,
    'prob_rf': probs_rf
})

In [27]:
print("--- 10 самых токсичных текстов (Naive Bayes + CountVectorizer) ---")
top_10_nb = results_df.sort_values(by='prob_nb', ascending=False).head(10)
print(top_10_nb[['text', 'prob_nb', 'true_label']])

--- 10 самых токсичных текстов (Naive Bayes + CountVectorizer) ---
                                                                                                  text  \
1405                          I can't believe these women, thinking I won't murder every one of them.    
473                           The life of any insect is more valuable than that of a disabled person.    
1530                     Some immigrants get into government. They are still just common trash to me.    
1502            An immigrant must have committed that crime. Only they can be that violent and cruel.    
1523          Immigrants can be kind to you at times. Don't forget they're still just worthless scum.    
1598  We're talking about disabled people here. They can't help themselves but be complete dickheads.    
1590                           They are all complete assholes. Needless to say they are black people.    
1434                        Trans people are never going to change so we should just 

In [28]:
print("--- 10 самых токсичных текстов (Random Forest + TfidfVectorizer) ---")
top_10_rf = results_df.sort_values(by='prob_rf', ascending=False).head(10)
print(top_10_rf[['text', 'prob_rf', 'true_label']])

--- 10 самых токсичных текстов (Random Forest + TfidfVectorizer) ---
                                      text   prob_rf  true_label
906           You are nothing but a slut.   0.972599           1
3660         You are nothing but a b1tch.   0.972350           1
917           You are nothing but a mong.   0.972350           1
3672          You are nothing but a m0ng.   0.972350           1
914           You are nothing but a coon.   0.972350           1
913         You are nothing but a nigger.   0.972350           1
909        You are nothing but a shemale.   0.972350           1
3675   You are nothing but a camelfuck3r.   0.972350           1
920   You are nothing but a camel fucker.   0.972350           1
3668        You are nothing but a nigg3r.   0.972350           1


Прежде всего я хочу сказать, что датасет выглядит крайне искусственно. Те комментарии, которые мы увидели, слишком гротескно-провокационные, едва ли в живом Интернете их в таком виде можно встретить даже у законченных фашистов. Отсюда и очень высокие метрики у обоих классификаторов.

Мы видим, что наивный байесовский классификатор присваивает вероятность 1 всем нашим топ-10, подозреваю, что и дальше таких тоже очень много — поэтому нельзя сказать, что это "самые токсичные по мнению классификатора" тексты — таких очень много. Но в топе мы наблюдаем просто преступные и расистские хейт-высказывания.

Рандомный лес выдаёт более разнообразные вероятности, и в топе оказались, по сути, индивидуальные оскорбления «you are nothing but ...», которые, конечно, являются хейтом, но всё же сомневаюсь, что этот показатель у них должен быть выше, чем у высказываний с ненавистью по отношению ко всей группе. Учитывая более низкий Recall для нулевого класса, видимо, имеет место переобучение и концентрация на таких словах, как *you* и *nothing*, ну и, наверное, на названиях самих групп.

Прямых миссклассификаций ни одна модель в топе не допустила.

Так что скорее выходит, что с таким простым и понятным, но искусственным и малополезным в реальности датасеты немного лучше справляются более простые классификаторы, а при усложнениях уже возможен риск переобучения.

## Задание 3 (2 балла - 0,5 балл за каждый классификатор)

Для классификаторов Logistic Regression, Decision Trees, Naive Bayes, RandomForest найдите способ извлечь важность признаков для предсказания токсичного класса (любой из датасетов). Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов. 

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию. 
Также как и в предыдущем задании у классификаторов должно быть задано вручную как минимум 2 параметра (по возможности, f1 мера каждого из классификаторов должна быть минимум 0.6)

Возьмём датасет из первого задания, и попробуем улучшить наш результат с мешком слов+Mystem. Будем использовать TF-IDF и триграммы, ну и лемматизацию, конечно же. Мы попробуем варианты с удалением стоп-слов и без него, так как мы увидели, что логистическая регрессия со стоп-словами перформит хуже, чем без них — проверим это и на других классификаторах.

In [10]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

### 1. Функция для извлечения и отображения топ-N фичей

In [11]:
def show_top_features(pipeline, top_n=10):
    # Извлекаем обученный векторизатор и классификатор
    vectorizer = pipeline.named_steps['vect']
    classifier = pipeline.named_steps['clf']

    # Получаем имена фичей
    feature_names = vectorizer.get_feature_names_out()

    coef_data = None
    top_indices = None

    # Логика для логистической регрессии
    if hasattr(classifier, 'coef_'):
        print(f"Top {top_n} токсичных фичей:")

        if classifier.coef_.shape[0] == 1:
            coef_data = classifier.coef_[0]
        else:
            coef_data = classifier.coef_[1]

        # Сортируем по убыванию (самые токсичные слова)
        top_indices = np.argsort(coef_data)[-top_n:][::-1]

    # Логика для деревьев и леса
    elif hasattr(classifier, 'feature_importances_'):
        print(
            f"Top {top_n} важных фичей:")
        coef_data = classifier.feature_importances_

        # Сортируем по убыванию (самые важные слова)
        top_indices = np.argsort(coef_data)[-top_n:][::-1]

    # Логика для Naive Bayes
    elif hasattr(classifier, 'feature_log_prob_'):
        print(f"Top {top_n} токсичных фичей:")

        # Рассчитываем важность как log(P(word|toxic)) - log(P(word|not_toxic))
        coef_data = classifier.feature_log_prob_[
            1] - classifier.feature_log_prob_[0]

        # Сортируем по убыванию (самые токсичные слова)
        top_indices = np.argsort(coef_data)[-top_n:][::-1]

    else:
        print(
            f"Этот классификатор ({type(classifier).__name__}) не имеет 'coef_', 'feature_importances_' или 'feature_log_prob_'.")
        return

    # Выводим результат
    if top_indices is not None:
        for i in top_indices:
            print(f"  {feature_names[i]}: {coef_data[i]:.4f}")

### 2. Параметры векторизатора (TF-IDF)

In [12]:
tfidf_params = {
    'tokenizer': tokenize_mystem,
    'stop_words': None,
    'ngram_range': (1, 3),
    'min_df': 3
}

### 3. Логрег

In [13]:
print("--- Logistic Regression ---")

pipeline_lr = Pipeline([
    ('vect', TfidfVectorizer(**tfidf_params)),
    ('clf', LogisticRegression(
        random_state=42,
        solver='liblinear',
        class_weight='balanced',
        C=5.0
    ))
])

--- Logistic Regression ---


In [14]:
pipeline_lr.fit(X_train, y_train)

preds_lr = pipeline_lr.predict(X_test)
print(classification_report(y_test, preds_lr))

/home/futyn/miniconda3/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


              precision    recall  f1-score   support

         0.0       0.90      0.92      0.91      1918
         1.0       0.84      0.80      0.82       965

    accuracy                           0.88      2883
   macro avg       0.87      0.86      0.87      2883
weighted avg       0.88      0.88      0.88      2883



In [15]:
show_top_features(pipeline_lr, top_n=10)

Top 10 токсичных фичей:
  ты: 9.4352
  хохол: 9.2681
  тупой: 7.8799
  хохлов: 7.5105
  шлюха: 7.0121
  дебил: 6.9429
  русский: 6.7398
  быдло: 6.0374
  баба: 5.4191
  долбоеб: 5.2284


> Весь русский Твиттер в десяти словах — фото в цвете

### 4. Решающее дерево

In [16]:
print("--- Decision Tree ---")

pipeline_dt = Pipeline([
    ('vect', TfidfVectorizer(**tfidf_params)),
    ('clf', DecisionTreeClassifier(
        random_state=42,
        max_depth=25,
        min_samples_leaf=5,
        class_weight='balanced'
    ))
])

--- Decision Tree ---


In [17]:
pipeline_dt.fit(X_train, y_train)

preds_dt = pipeline_dt.predict(X_test)
print(classification_report(y_test, preds_dt))

/home/futyn/miniconda3/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


              precision    recall  f1-score   support

         0.0       0.83      0.68      0.75      1918
         1.0       0.54      0.72      0.62       965

    accuracy                           0.70      2883
   macro avg       0.68      0.70      0.68      2883
weighted avg       0.73      0.70      0.71      2883



In [18]:
show_top_features(pipeline_dt, top_n=10)

Top 10 важных фичей:
  ты: 0.1665
  быть: 0.0617
  но: 0.0411
  хохол: 0.0397
  если: 0.0320
  в: 0.0314
  русский: 0.0298
  не: 0.0287
  он: 0.0246
  год: 0.0217


### 5. Naive Bayes

In [19]:
print("--- Multinomial Naive Bayes ---")

pipeline_nb = Pipeline([
    ('vect', TfidfVectorizer(**tfidf_params)),
    ('clf', MultinomialNB(
        alpha=0.01,
        fit_prior=False
    ))
])

--- Multinomial Naive Bayes ---


In [20]:
pipeline_nb.fit(X_train, y_train)

preds_nb = pipeline_nb.predict(X_test)
print(classification_report(y_test, preds_nb))

/home/futyn/miniconda3/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


              precision    recall  f1-score   support

         0.0       0.89      0.92      0.91      1918
         1.0       0.83      0.79      0.81       965

    accuracy                           0.87      2883
   macro avg       0.86      0.85      0.86      2883
weighted avg       0.87      0.87      0.87      2883



In [21]:
show_top_features(pipeline_nb, top_n=10)

Top 10 токсичных фичей:
  хохлов: 8.4636
  дегенерат: 7.9276
  пидорашек: 7.6397
  даун: 7.6260
  хуесос: 7.6253
  рашек: 7.4731
  пидораха: 7.4195
  шизик: 7.3858
  жид: 7.3403
  лахта: 7.3133


### 6. Лес

In [22]:
print("\n--- Random Forest ---")

pipeline_rf = Pipeline([
    ('vect', TfidfVectorizer(**tfidf_params)),
    ('clf', RandomForestClassifier(
        random_state=42,
        n_estimators=200,
        max_depth=35,
        class_weight='balanced',
        n_jobs=-1
    ))
])


--- Random Forest ---


In [23]:
pipeline_rf.fit(X_train, y_train)

preds_rf = pipeline_rf.predict(X_test)
print(classification_report(y_test, preds_rf))

/home/futyn/miniconda3/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


              precision    recall  f1-score   support

         0.0       0.87      0.81      0.84      1918
         1.0       0.67      0.76      0.71       965

    accuracy                           0.79      2883
   macro avg       0.77      0.79      0.78      2883
weighted avg       0.80      0.79      0.80      2883



In [24]:
show_top_features(pipeline_rf, top_n=10)

Top 10 важных фичей:
  ты: 0.0490
  быть: 0.0168
  в: 0.0125
  хохол: 0.0119
  но: 0.0118
  не: 0.0108
  год: 0.0098
  если: 0.0091
  можно: 0.0085
  русский: 0.0083


Получилось, что дерево и лес деревьев показали худший результат, спотыкаясь о стоп-слова, а логрегу и наивному байесовскому классификатору они как будто особо не мешают. Посмотрим, что будет, если их убрать.

In [25]:
tfidf_params_stop = {
    'tokenizer': tokenize_mystem,
    'stop_words': russian_stopwords,
    'ngram_range': (1, 3),
    'min_df': 3
}

### 7. Логрег (стоп-слова)

In [26]:
print("--- Logistic Regression ---")

pipeline_lr = Pipeline([
    ('vect', TfidfVectorizer(**tfidf_params_stop)),
    ('clf', LogisticRegression(
        random_state=42,
        solver='liblinear',
        class_weight='balanced',
        C=5.0
    ))
])

--- Logistic Regression ---


In [27]:
pipeline_lr.fit(X_train, y_train)

preds_lr = pipeline_lr.predict(X_test)
print(classification_report(y_test, preds_lr))

/home/futyn/miniconda3/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/futyn/miniconda3/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['весь', 'свой', 'это'] not in stop_words.
  warnings.warn(


              precision    recall  f1-score   support

         0.0       0.90      0.91      0.90      1918
         1.0       0.81      0.79      0.80       965

    accuracy                           0.87      2883
   macro avg       0.85      0.85      0.85      2883
weighted avg       0.87      0.87      0.87      2883



In [28]:
show_top_features(pipeline_lr, top_n=10)

Top 10 токсичных фичей:
  хохол: 8.2803
  дебил: 7.7176
  тупой: 7.3355
  хохлов: 6.9765
  шлюха: 6.4680
  быдло: 6.0431
  сука: 5.3789
  русский: 5.3706
  пидор: 5.2628
  долбоеб: 5.1829


### 8. Дерево (стоп-слова)

In [29]:
print("--- Decision Tree ---")

pipeline_dt = Pipeline([
    ('vect', TfidfVectorizer(**tfidf_params_stop)),
    ('clf', DecisionTreeClassifier(
        random_state=42,
        max_depth=25,
        min_samples_leaf=5,
        class_weight='balanced'
    ))
])

--- Decision Tree ---


In [30]:
pipeline_dt.fit(X_train, y_train)

preds_dt = pipeline_dt.predict(X_test)
print(classification_report(y_test, preds_dt))

/home/futyn/miniconda3/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/futyn/miniconda3/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['весь', 'свой', 'это'] not in stop_words.
  warnings.warn(


              precision    recall  f1-score   support

         0.0       0.84      0.46      0.59      1918
         1.0       0.43      0.83      0.57       965

    accuracy                           0.58      2883
   macro avg       0.64      0.64      0.58      2883
weighted avg       0.71      0.58      0.58      2883



In [31]:
show_top_features(pipeline_dt, top_n=10)

Top 10 важных фичей:
  хохол: 0.0711
  год: 0.0656
  твой: 0.0574
  очень: 0.0500
  тупой: 0.0484
  русский: 0.0464
  это: 0.0444
  свой: 0.0444
  работа: 0.0336
  ебать: 0.0333


### 9. Naive Bayes (стоп-слова)

In [32]:
print("--- Multinomial Naive Bayes ---")

pipeline_nb = Pipeline([
    ('vect', TfidfVectorizer(**tfidf_params_stop)),
    ('clf', MultinomialNB(
        alpha=0.01,
        fit_prior=False
    ))
])

--- Multinomial Naive Bayes ---


In [33]:
pipeline_nb.fit(X_train, y_train)

preds_nb = pipeline_nb.predict(X_test)
print(classification_report(y_test, preds_nb))

/home/futyn/miniconda3/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/futyn/miniconda3/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['весь', 'свой', 'это'] not in stop_words.
  warnings.warn(


              precision    recall  f1-score   support

         0.0       0.90      0.91      0.91      1918
         1.0       0.82      0.80      0.81       965

    accuracy                           0.87      2883
   macro avg       0.86      0.86      0.86      2883
weighted avg       0.87      0.87      0.87      2883



In [34]:
show_top_features(pipeline_nb, top_n=10)

Top 10 токсичных фичей:
  хохлов: 8.8064
  дегенерат: 8.2056
  пидорашек: 7.8849
  даун: 7.8005
  рашек: 7.7748
  хуесос: 7.7710
  шизик: 7.7162
  пидораха: 7.6865
  жид: 7.5449
  малолетний: 7.5230


### 10. Лес

In [35]:
print("\n--- Random Forest ---")

pipeline_rf = Pipeline([
    ('vect', TfidfVectorizer(**tfidf_params_stop)),
    ('clf', RandomForestClassifier(
        random_state=42,
        n_estimators=200,
        max_depth=35,
        class_weight='balanced',
        n_jobs=-1
    ))
])


--- Random Forest ---


In [36]:
pipeline_rf.fit(X_train, y_train)

preds_rf = pipeline_rf.predict(X_test)
print(classification_report(y_test, preds_rf))

/home/futyn/miniconda3/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/futyn/miniconda3/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['весь', 'свой', 'это'] not in stop_words.
  warnings.warn(


              precision    recall  f1-score   support

         0.0       0.88      0.77      0.82      1918
         1.0       0.64      0.80      0.71       965

    accuracy                           0.78      2883
   macro avg       0.76      0.78      0.76      2883
weighted avg       0.80      0.78      0.78      2883



In [37]:
show_top_features(pipeline_rf, top_n=10)

Top 10 важных фичей:
  хохол: 0.0208
  год: 0.0195
  твой: 0.0184
  очень: 0.0169
  тупой: 0.0160
  русский: 0.0145
  хохлов: 0.0122
  свой: 0.0110
  ебать: 0.0104
  работать: 0.0101


Итак, выходит, что логрег и наивный байесовский классификатор смотрят на те слова, на которые действительно нужно смотреть при учёте токсичности. Снижение качества логрега при удалении стоп-слов объясняется важностью слова *ты*, которое, видимо, в этих комментариях постоянно сопровождается оскорблениями в адрес собеседника.

Дерево и лес деревьев выводят странные корреляции из данных и перформят сильно хуже, в топ-фичах хоть и есть релевантные слова, но их сильно меньше — много просто частотных слов, и даже удаление стоп-слов проблему не сильно решает, а результат становится только хуже. По-видимому, эти модели плохо работают с важностью слов, здесь вероятностные, конечно, лучше.

Ну и ещё надо сказать, что никакая модель не уделяет достаточной важности Н-граммам, в топе их нет — видимо, всё же отдельные слова сильно красноречивее и их уже достаточно.

# Задание 4* (2 балла)

Составьте ансамблевый классификатор вручную. Используя один из парных датасетов в `glue` (например, `mnli`) обучите как минимум 10 разных классификаторов (так как количество алгоритмов меньше 10, используйте разные комбинации параметров в векторайзерах и в классификаторах, например, tfidf + logreg и countvectorizer+logreg). Чем сильнее каждый классификатор отличается от другого, тем лучше. 

Вместо стандартного разбиения на train и test, разбейте выборку на 3 части: train, dev и test. Используйте train для обучения 10 классификаторов. Сделайте предсказания всеми классификаторами для dev и test датасетов. Используя объединенные предсказания всех классификаторов на dev части как признаки, обучите еще один общий классификатор. Сделайте предсказания общим классификатором на test части (опять же используйте предсказания 10 классификаторов как признаки) и рассчитайте качество итоговой классификации. 

Также отдельно оцените качество на test части для каждого из 10 классификаторов. Сравните с общей оценкой. Превосходит ли общий классификатор в качестве самый лучший из отдельных классификаторов? 

Сейчас здесь будет много-много ячеек, потому что для каждого из десяти классификаторов нужно написать свой пайплайн, а потом это ещё и соединить. Спасибо ColumnTransformer, который может хэндлить парные тексты, а то бы их было ещё больше.

In [2]:
import pandas as pd
import numpy as np
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

### 1. Загрузка данных и подготовка сплитов

In [3]:
nli_dataset = load_dataset('nyu-mll/glue', 'mnli')

print(nli_dataset)

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})


In [4]:
# Возьмем 10000 примеров из 'train' для обучения и 2000 из 'validation' для теста
# (иначе мы тут долго будем сидеть)
data = nli_dataset['train'].shuffle(seed=42).select(range(10000))
test_data = nli_dataset['validation_matched'].shuffle(
    seed=42).select(
        range(2000))

# Конвертируем в pandas для удобства
data_df = pd.DataFrame(data)
test_df = pd.DataFrame(test_data)

In [5]:
# Разделяем 10000 примеров на 8000 (train) и 2000 (dev)
train_df, dev_df = train_test_split(
    data_df, test_size=0.2, random_state=42, stratify=data_df['label'])

In [6]:
X_train = train_df[['premise', 'hypothesis']]
y_train = train_df['label']

X_dev = dev_df[['premise', 'hypothesis']]
y_dev = dev_df['label']

X_test = test_df[['premise', 'hypothesis']]
y_test = test_df['label']

print(f"Размер train: {len(X_train)}")
print(f"Размер dev: {len(X_dev)}")
print(f"Размер test: {len(X_test)}")

Размер train: 8000
Размер dev: 2000
Размер test: 2000


### 2. Векторизаторы и модельки

У нас будет три препроцессора и десять разных пайплайнов, которые будут комбинировать эти препроцессоры и разные классификаторы

In [7]:
# Векторизация 1: TF-IDF (1-2 граммы, 1000 фичей)
preprocessor_1 = ColumnTransformer([
    ('premise_tfidf', TfidfVectorizer(max_features=1000, ngram_range=(1, 2)), 'premise'),
    ('hypothesis_tfidf', TfidfVectorizer(max_features=1000, ngram_range=(1, 2)), 'hypothesis')
], remainder='drop')

# Векторизация 2: CountVectorizer (только слова, 1000 фичей)
preprocessor_2 = ColumnTransformer([
    ('premise_count', CountVectorizer(max_features=1000, ngram_range=(1, 1)), 'premise'),
    ('hypothesis_count', CountVectorizer(max_features=1000, ngram_range=(1, 1)), 'hypothesis')
], remainder='drop')

# Векторизация 3: TF-IDF (1-3 граммы, все фичи, другие min/max_df)
preprocessor_3 = ColumnTransformer([
    ('premise_tfidf_ng', TfidfVectorizer(ngram_range=(1, 3), min_df=5, max_df=0.5), 'premise'),
    ('hypothesis_tfidf_ng', TfidfVectorizer(ngram_range=(1, 3), min_df=5, max_df=0.5), 'hypothesis')
], remainder='drop')

In [8]:
# Наши базовые модели

models = {}

# Модели на preprocessor_1
models['model_1_tfidf_lr'] = Pipeline(
    # Логрег
    [('prep', preprocessor_1), ('clf', LogisticRegression(C=1.0, random_state=42, max_iter=1000))])
models['model_2_tfidf_nb'] = Pipeline(
    # Наивный байесовский
    [('prep', preprocessor_1), ('clf', MultinomialNB(alpha=1.0))])
# Лес
models['model_3_tfidf_rf'] = Pipeline([('prep',
                                        preprocessor_1),
                                       ('clf',
                                        RandomForestClassifier(n_estimators=100,
                                                               max_depth=10,
                                                               random_state=42))])
models['model_4_tfidf_lr_weak'] = Pipeline(
    # Логрег с другим C
    [('prep', preprocessor_1), ('clf', LogisticRegression(C=0.1, random_state=42, max_iter=1000))])


# Модели на preprocessor_2
models['model_5_count_lr'] = Pipeline(
    # Логрег
    [('prep', preprocessor_2), ('clf', LogisticRegression(C=1.0, random_state=42, max_iter=1000))])
models['model_6_count_nb'] = Pipeline(
    # Наивный байесовский
    [('prep', preprocessor_2), ('clf', MultinomialNB(alpha=1.0))])
# Лес
models['model_7_count_rf'] = Pipeline([('prep',
                                        preprocessor_2),
                                       ('clf',
                                        RandomForestClassifier(n_estimators=50,
                                                               max_depth=5,
                                                               random_state=42))])

# Модели на preprocessor_3
# Логрег
models['model_8_tfidf_ng_lr'] = Pipeline([('prep', preprocessor_3), ('clf', LogisticRegression(
    C=5.0, class_weight='balanced', random_state=42, max_iter=1000))])
models['model_9_tfidf_ng_nb'] = Pipeline(
    # Наивный байесовский
    [('prep', preprocessor_3), ('clf', MultinomialNB(alpha=0.1))])
# Лес
models['model_10_tfidf_ng_rf'] = Pipeline([('prep', preprocessor_3), ('clf', RandomForestClassifier(
    n_estimators=200, max_depth=20, class_weight='balanced', random_state=42, n_jobs=-1))])

### 3. Обучение базовых моделей и создание фич

In [9]:
# Списки для хранения фич
dev_meta_features = []
test_meta_features = []

# Обучаем базовые модели и получаем предсказания
for name, model in models.items():
    print(f"Обучение {name}...")

    # Обучаем на train
    model.fit(X_train, y_train)

    # Получаем вероятности для dev
    dev_preds = model.predict_proba(X_dev)
    dev_meta_features.append(dev_preds)

    # 3. Получаем вероятности для test
    test_preds = model.predict_proba(X_test)
    test_meta_features.append(test_preds)

Обучение model_1_tfidf_lr...
Обучение model_2_tfidf_nb...
Обучение model_3_tfidf_rf...
Обучение model_4_tfidf_lr_weak...
Обучение model_5_count_lr...
Обучение model_6_count_nb...
Обучение model_7_count_rf...
Обучение model_8_tfidf_ng_lr...
Обучение model_9_tfidf_ng_nb...
Обучение model_10_tfidf_ng_rf...


In [10]:
# Собираем фичи в единую матрицу

X_dev_meta = np.hstack(dev_meta_features)
X_test_meta = np.hstack(test_meta_features)

print(f"размер матрицы мета-признаков (dev): {X_dev_meta.shape}")
print(f"Размер матрицы мета-признаков (test): {X_test_meta.shape}")

размер матрицы мета-признаков (dev): (2000, 30)
Размер матрицы мета-признаков (test): (2000, 30)


### 4. Обучаем итоговый классификатор на полученных фичах

Будем использовать логрег, потому что он хорошо подходит для взвешивания предсказаний (как вероятностная модель)

In [11]:
meta_classifier = LogisticRegression(random_state=42, max_iter=2000)

print("--- Обучение классификатора ---")
meta_classifier.fit(X_dev_meta, y_dev)

--- Обучение классификатора ---


LogisticRegression(max_iter=2000, random_state=42)

In [12]:
print("--- Финальная оценка")

final_preds = meta_classifier.predict(X_test_meta)
print(classification_report(y_test, final_preds, zero_division=0))

--- Финальная оценка
              precision    recall  f1-score   support

           0       0.45      0.47      0.46       695
           1       0.43      0.43      0.43       650
           2       0.50      0.47      0.48       655

    accuracy                           0.46      2000
   macro avg       0.46      0.46      0.46      2000
weighted avg       0.46      0.46      0.46      2000



### 5. Смотрим на метрики отдельных классификаторов и сравниваем

In [15]:
# Список для хранения всех метрик
all_metrics = []

# Оценка базовых моделей
for name, model in models.items():
    base_preds = model.predict(X_test)

    report = classification_report(
        y_test,
        base_preds,
        output_dict=True,
        zero_division=0)

    all_metrics.append({
        'model': name,
        'accuracy': report['accuracy'],
        'precision (w_avg)': report['weighted avg']['precision'],
        'recall (w_avg)': report['weighted avg']['recall'],
        'f1 (w_avg)': report['weighted avg']['f1-score']
    })

# Оценка мета-модели

final_preds = meta_classifier.predict(X_test_meta)
report_meta = classification_report(
    y_test,
    final_preds,
    output_dict=True,
    zero_division=0)

all_metrics.append({
    'model': 'META_CLASSIFIER',
    'accuracy': report_meta['accuracy'],
    'precision (w_avg)': report_meta['weighted avg']['precision'],
    'recall (w_avg)': report_meta['weighted avg']['recall'],
    'f1 (w_avg)': report_meta['weighted avg']['f1-score']
})

# Делаем красиво
metrics_df = pd.DataFrame(all_metrics)
metrics_df = metrics_df.sort_values(by='f1 (w_avg)', ascending=False)

print(metrics_df.to_string(index=False))

                model  accuracy  precision (w_avg)  recall (w_avg)  f1 (w_avg)
model_4_tfidf_lr_weak    0.4655           0.469361          0.4655    0.465878
 model_10_tfidf_ng_rf    0.4585           0.471544          0.4585    0.458784
      META_CLASSIFIER    0.4585           0.459306          0.4585    0.458700
     model_7_count_rf    0.4575           0.472169          0.4575    0.457485
     model_3_tfidf_rf    0.4480           0.463160          0.4480    0.447723
     model_1_tfidf_lr    0.4460           0.446548          0.4460    0.446193
     model_2_tfidf_nb    0.4435           0.444784          0.4435    0.443608
     model_5_count_lr    0.4415           0.441914          0.4415    0.441658
  model_8_tfidf_ng_lr    0.4385           0.440049          0.4385    0.439058
     model_6_count_nb    0.4175           0.418963          0.4175    0.416867
  model_9_tfidf_ng_nb    0.4155           0.416717          0.4155    0.415576


Видим, что наш обученный на признаках классификатор не перформит лучше отдельных моделей - ансамблирование здесь не сработало. С другой стороны, перформит он не сильно хуже моделей-лидеров, хотя все они перформят плохо.

Во-первых, надо заметить, что наш общий классификатор учился на 2000 датапойнтах, тогда как отдельные классификаторы — на 8000. Наверное, учитывая это, результат не настолько плохой: мы не получили модельку, которая перформит лучше, но она легче и быстрее и требует для предсказания вектор размером 30. Возможно, если расширить валидационный сет, может быть лучше.

Во-вторых, задача NLI всё же сложная для BoW-моделей, да и сами эти модели не сильно различаются между собой. Да, разные параметры, разные векторы, но на практике всё тот же BoW и ничего особенно нового мы не получили.

Поэтому техника, конечно, интересная, но требует разных и более различающихся моделей: например, ансамблирования BoW и какого-нибудь W2V в разных комбинациях. А так — garbage in, garbage out.